In [9]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"

pickle_path = "../../data/tencent2020/pickle"
if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 

In [2]:
train_click_log = pd.read_csv(path_save + "train_click_log.csv", encoding="utf-8")
test_click_log = pd.read_csv(path_save + "test_click_log.csv", encoding="utf-8")
train_click_log.shape, test_click_log.shape

((30082771, 9), (33585512, 9))

train：user_id 范围 (1, 900000)  
test ：user_id 范围 (3000001, 4000000) 

In [3]:
train_click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2035918,1261,2,6783,6
3,6,309204,325532,1,292523,27081,3,32066,242
4,59,309204,2746730,1,2362208,0,18,14682,88


In [4]:
test_click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,3131989,645764,1,573314,58,2,14689,6
1,20,3131989,1027422,1,902764,129,2,42272,6
2,20,3131989,1106443,1,970829,2171,2,37513,322
3,20,3131989,629802,1,559183,0,18,14678,26
4,59,3131989,2839769,1,2441288,129,2,35328,6


In [6]:
click_log = train_click_log.append(test_click_log)
click_log.sort_values(by=['user_id','time'], ascending=[True,True], inplace=True)
click_log = click_log.reset_index(drop=True)
click_log.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,1,821396,1,724607,0,5,7293,326
1,20,1,209778,1,188507,136,2,9702,6
2,20,1,877468,1,773445,0,5,29455,106
3,39,1,1683713,1,1458878,0,5,14668,326
4,40,1,122032,1,109959,1334,2,11411,0


In [10]:
user_creative_ids = click_log.groupby(['user_id'])['creative_id'].apply(lambda x:list(x)).reset_index()
user_creative_ids.to_pickle(f"{pickle_path}/user_creative_ids.pickle")
user_creative_ids.head()

,user_id,creative_id
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169..."
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ..."
2,3,"[661347, 808612, 710859, 825434, 593522, 72694..."
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264..."
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434..."
